In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Selenium으로 유튜브 댓글 페이지 접속
def open_youtube_comments(video_url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 화면을 띄우지 않음
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    
    # Chrome 드라이버 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get(video_url)
    time.sleep(3)

    # 페이지 스크롤을 끝까지 내려 댓글을 로드
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    scroll_attempts = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_height == last_height:
            scroll_attempts += 1
            if scroll_attempts > 2:  # 동일한 높이에서 스크롤이 멈추면 중지
                break
        else:
            scroll_attempts = 0  # 스크롤이 성공적으로 내려갔으면 리셋
        last_height = new_height

    time.sleep(2)
    return driver  # driver 반환

# 사용 예시
if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=m_o0MpnD7oQ"  # 유튜브 영상 URL 입력
    driver = open_youtube_comments(video_url)
    # 다음 단계로 넘어가기 전에 driver를 사용할 수 있습니다.


In [8]:
# 대댓글 버튼 모두 클릭하기
def click_more_replies(driver):
    buttons = driver.find_elements(By.CSS_SELECTOR, "#more-replies > a")
    for button in buttons:
        try:
            button.send_keys(Keys.ENTER)
            time.sleep(1.5)
            button.click()
        except:
            pass

# 사용 예시
if __name__ == "__main__":
    # 이전 단계에서 반환된 driver를 사용
    click_more_replies(driver)
    # 다음 단계로 넘어가기 전에 driver를 사용할 수 있습니다.


In [9]:
from bs4 import BeautifulSoup

# BeautifulSoup으로 HTML 파싱 및 댓글/대댓글 추출
def get_comments_from_html(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # 작성자와 댓글 내용 추출
    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    id_final = []
    comment_final = []

    for i in range(len(comment_list)):
        temp_id = id_list[i].text.strip().replace('\n', '').replace('\t', '').replace('    ', '')
        temp_comment = comment_list[i].text.strip().replace('\n', '').replace('\t', '').replace('    ', '')
        
        id_final.append(temp_id)  # 댓글 작성자
        comment_final.append(temp_comment)  # 댓글 내용

    return id_final, comment_final

# 사용 예시
if __name__ == "__main__":
    # 이전 단계에서 반환된 driver를 사용
    id_final, comment_final = get_comments_from_html(driver)
    # 다음 단계로 넘어가기 전에 id_final과 comment_final을 사용할 수 있습니다.


In [10]:
import pandas as pd

# 댓글을 엑셀로 저장하는 함수
def save_comments_to_excel(id_final, comment_final, file_name):
    if id_final and comment_final:
        pd_data = {"아이디": id_final, "댓글 내용": comment_final}
        youtube_pd = pd.DataFrame(pd_data)
        
        # 엑셀 파일로 저장
        youtube_pd.to_excel(file_name, index=False)
        print(f"댓글이 {file_name}에 저장되었습니다.")
    else:
        print("저장할 댓글이 없습니다.")

# 사용 예시
if __name__ == "__main__":
    file_name = "youtube_comments.xlsx"  # 저장할 엑셀 파일 이름
    save_comments_to_excel(id_final, comment_final, file_name)


저장할 댓글이 없습니다.


In [11]:
if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=m_o0MpnD7oQ"  # 유튜브 영상 URL 입력
    driver = open_youtube_comments(video_url)
    click_more_replies(driver)
    id_final, comment_final = get_comments_from_html(driver)
    file_name = "youtube_comments.xlsx"  # 저장할 엑셀 파일 이름
    save_comments_to_excel(id_final, comment_final, file_name)
    driver.quit()  # 드라이버 종료


저장할 댓글이 없습니다.
